In [1]:
import pandas as pd
import numpy as np
import ast
import torch
import os

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from typing import Any

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Log into HuggingFace

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from huggingface_hub import login

hf_token = os.environ["HF_TOKEN"]
login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/khalidrajan/.cache/huggingface/token
Login successful


# Load and Clean Dataset

In [6]:
dataset_name = "darrow-ai/LegalLensNER-SharedTask"

In [7]:
from datasets import load_dataset
dataset = load_dataset(dataset_name)

In [8]:
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Return None
        return None

In [9]:
# convert string representations of lists into actual lists
dataset = dataset.map(
    lambda x: {
        "tokens": safe_literal_eval(x["tokens"]),
        "ner_tags": safe_literal_eval(x["ner_tags"]),
    }
)

In [10]:
def is_not_none(example):
    # Check that 'tokens' and 'ner_tags' fields are not None
    return example["tokens"] is not None and example["ner_tags"] is not None

# Filter the dataset to remove examples with None values in 'tokens' or 'ner_tags'
dataset = dataset.filter(is_not_none)

In [11]:
def non_matching_len(example):
    # Check that the len of tokens matches the len of NER tags
    return len(example["tokens"]) == len(example["ner_tags"])

dataset = dataset.filter(non_matching_len)

In [12]:
# split dataset into train and eval sets
dataset = dataset["train"].train_test_split(test_size=0.3, seed=1234)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 679
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 291
    })
})

In [14]:
def get_label_list(dataset: Dataset):
    label_set = set()
    for data in dataset:
        labels = data["ner_tags"]
        label_set.update(labels)
    return list(label_set)

In [15]:
label_list = get_label_list(
    dataset["train"]
)  # Assuming 'train' split exists and contains the labels

In [16]:
label_list

['O',
 'B-VIOLATION',
 'I-VIOLATED ON',
 'B-VIOLATED BY',
 'I-VIOLATED BY',
 'B-VIOLATED ON',
 'B-LAW',
 'I-LAW',
 'I-VIOLATION']

In [17]:
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

# Tokenize Dataset

In [18]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
    )
    labels = []
    for i, example_labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        last_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != last_word_id:
                label_id = label_to_id.get(example_labels[word_id], -100)
                label_ids.append(label_id)
            else:
                label_ids.append(label_id)
            last_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
model_checkpoint = "FacebookAI/roberta-base"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True, use_fast = True)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
tokenized_dataset = dataset.map(
    lambda x: tokenize_and_align_labels(x, tokenizer), batched=True
)

Map: 100%|██████████| 291/291 [00:00<00:00, 4009.14 examples/s]


# Train Model

In [22]:
import evaluate

metric = evaluate.load("seqeval")

In [23]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results


In [24]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label = id_to_label, label2id = label_to_id)

training_args = TrainingArguments(
                        output_dir=f"{model_checkpoint}_legal_ner_finetuned",
                        evaluation_strategy = "epoch",
                        save_strategy="epoch",
                        learning_rate = 2e-5,
                        num_train_epochs=10,
                        warmup_steps=500,
                        weight_decay=0.01)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
# Train the model
trainer.train()

 10%|█         | 85/850 [01:17<11:09,  1.14it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                
 10%|█         | 85/850 [01:26<11:09,  1.14it/s]

{'eval_loss': 0.7163046598434448, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 107, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 71, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 64, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 374, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.7706214689265537, 'eval_runtime': 9.6732, 'eval_samples_per_second': 30.083, 'eval_steps_per_second': 3.825, 'epoch': 1.0}


 20%|██        | 170/850 [02:46<09:50,  1.15it/s] /Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
 20%|██        | 170/850 [02:56<09:50,  1.15it/s]

{'eval_loss': 0.37249431014060974, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 107, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 71, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 64, 'eval_VIOLATION_precision': 0.25389755011135856, 'eval_VIOLATION_recall': 0.3048128342245989, 'eval_VIOLATION_f1': 0.27703523693803156, 'eval_VIOLATION_number': 374, 'eval_overall_precision': 0.25389755011135856, 'eval_overall_recall': 0.18506493506493507, 'eval_overall_f1': 0.21408450704225349, 'eval_overall_accuracy': 0.8888233849177106, 'eval_runtime': 9.7226, 'eval_samples_per_second': 29.93, 'eval_steps_per_second': 3.806, 'epoch': 2.0}


 30%|███       | 255/850 [04:14<08:24,  1.18it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
 30%|███       | 255/850 [04:23<08:24,  1.18it/s]

...

                                                 
100%|██████████| 850/850 [14:13<00:00,  1.20it/s]

{'eval_loss': 0.25197145342826843, 'eval_LAW_precision': 0.8773584905660378, 'eval_LAW_recall': 0.8691588785046729, 'eval_LAW_f1': 0.8732394366197183, 'eval_LAW_number': 107, 'eval_VIOLATED BY_precision': 0.8225806451612904, 'eval_VIOLATED BY_recall': 0.7183098591549296, 'eval_VIOLATED BY_f1': 0.7669172932330828, 'eval_VIOLATED BY_number': 71, 'eval_VIOLATED ON_precision': 0.53125, 'eval_VIOLATED ON_recall': 0.53125, 'eval_VIOLATED ON_f1': 0.53125, 'eval_VIOLATED ON_number': 64, 'eval_VIOLATION_precision': 0.6624685138539043, 'eval_VIOLATION_recall': 0.7032085561497327, 'eval_VIOLATION_f1': 0.682230869001297, 'eval_VIOLATION_number': 374, 'eval_overall_precision': 0.7011128775834659, 'eval_overall_recall': 0.7159090909090909, 'eval_overall_f1': 0.708433734939759, 'eval_overall_accuracy': 0.9545566199950872, 'eval_runtime': 9.0247, 'eval_samples_per_second': 32.245, 'eval_steps_per_second': 4.1, 'epoch': 10.0}


100%|██████████| 850/850 [14:14<00:00,  1.01s/it]

{'train_runtime': 854.6281, 'train_samples_per_second': 7.945, 'train_steps_per_second': 0.995, 'train_loss': 0.3191476922876695, 'epoch': 10.0}


TrainOutput(global_step=850, training_loss=0.3191476922876695, metrics={'train_runtime': 854.6281, 'train_samples_per_second': 7.945, 'train_steps_per_second': 0.995, 'total_flos': 1774317261588480.0, 'train_loss': 0.3191476922876695, 'epoch': 10.0})

In [26]:
# evaluate model on eval set
evaluate_results = trainer.evaluate()

100%|██████████| 37/37 [00:08<00:00,  4.19it/s]


In [27]:
evaluate_results

{'eval_loss': 0.25197145342826843,
 'eval_LAW_precision': 0.8773584905660378,
 'eval_LAW_recall': 0.8691588785046729,
 'eval_LAW_f1': 0.8732394366197183,
 'eval_LAW_number': 107,
 'eval_VIOLATED BY_precision': 0.8225806451612904,
 'eval_VIOLATED BY_recall': 0.7183098591549296,
 'eval_VIOLATED BY_f1': 0.7669172932330828,
 'eval_VIOLATED BY_number': 71,
 'eval_VIOLATED ON_precision': 0.53125,
 'eval_VIOLATED ON_recall': 0.53125,
 'eval_VIOLATED ON_f1': 0.53125,
 'eval_VIOLATED ON_number': 64,
 'eval_VIOLATION_precision': 0.6624685138539043,
 'eval_VIOLATION_recall': 0.7032085561497327,
 'eval_VIOLATION_f1': 0.682230869001297,
 'eval_VIOLATION_number': 374,
 'eval_overall_precision': 0.7011128775834659,
 'eval_overall_recall': 0.7159090909090909,
 'eval_overall_f1': 0.708433734939759,
 'eval_overall_accuracy': 0.9545566199950872,
 'eval_runtime': 9.1573,
 'eval_samples_per_second': 31.778,
 'eval_steps_per_second': 4.04,
 'epoch': 10.0}

## Push our fine tuned model to the HuggingFace hub

In [28]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin: 100%|██████████| 5.24k/5.24k [00:00<00:00, 45.1kB/s]



























































































































































model.safetensors: 100%|██████████| 496M/496M [00:28<00:00, 17.7MB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [00:28<00:00, 14.20s/it]


CommitInfo(commit_url='https://huggingface.co/khalidrajan/roberta-base_legal_ner_finetuned/commit/f5ec09e1ca25e0b86392573437c5dba40218ba8f', commit_message='End of training', commit_description='', oid='f5ec09e1ca25e0b86392573437c5dba40218ba8f', pr_url=None, pr_revision=None, pr_num=None)